In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/imdb-dataset/IMDB-Dataset.csv


In [2]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.1 MB/s eta 0:00:00


# **Importation des Bibliothèques**

Importation des bibliothèques nécessaires pour le traitement des données et l'entraînement du modèle.

In [3]:
# Importation des bibliothèques essentielles pour le traitement des données et le machine learning
import numpy as np
import torch
import pandas as pd
from datasets import Dataset
from sklearn.model_selection import train_test_split



# Bibliothèque pour l'évaluation des modèles
import evaluate


# **Chargement des données**


In [4]:
# Chemin le fichier
file_path = '/kaggle/input/imdb-dataset/IMDB-Dataset.csv'

# Charger les données depuis le fichier CSV
df = pd.read_csv(file_path)
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [5]:
# Conversion des étiquettes en format numérique
df['label'] = df['sentiment'].map({'positive': 1, 'negative': 0})
df.drop(['sentiment'], axis=1, inplace=True)
df.head()

,review,label
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


# **Nettoyage du texte**

Il est important de supprimer les balises HTML, les caractères spéciaux et de normaliser le texte.

In [6]:
import re
from bs4 import BeautifulSoup

def clean_text(text):
    # Remove HTML tags
    text = BeautifulSoup(text, "html.parser").get_text()

    # Replace specific contractions
    contractions = {
        "i'm": "i am",
        "couldn't": "could not",
        "it's": "it is",
        "wouldn't": "would not",
        "don't": "do not",
        "doesn't": "does not",
        "didn't": "did not",
        "won't": "will not",
        "can't": "cannot",
        "isn't": "is not",
        "aren't": "are not",
        "wasn't": "was not",
        "weren't": "were not",
        "hasn't": "has not",
        "haven't": "have not",
        "hadn't": "had not",
        "doesn't": "does not"
    }

    # Replace contractions in the text
    for contraction, replacement in contractions.items():
        text = re.sub(r'\b' + contraction + r'\b', replacement, text, flags=re.IGNORECASE)

    # Remove special characters
    text = re.sub(r"[^a-zA-Z0-9\s]", "", text)

    # Convert to lowercase
    text = text.lower()

    return text

# Appliquer le nettoyage sur la colonne 'text'
df["clean_text"] = df["review"].astype(str).apply(clean_text)

<ipython-input-6-d59095b271e7>:6: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  text = BeautifulSoup(text, "html.parser").get_text()


In [7]:
df.head()

,review,label,clean_text
0,One of the other reviewers has mentioned that ...,1,one of the other reviewers has mentioned that ...
1,A wonderful little production. <br /><br />The...,1,a wonderful little production the filming tech...
2,I thought this was a wonderful way to spend ti...,1,i thought this was a wonderful way to spend ti...
3,Basically there's a family where a little boy ...,0,basically theres a family where a little boy j...
4,"Petter Mattei's ""Love in the Time of Money"" is...",1,petter matteis love in the time of money is a ...


In [8]:
df.drop(columns=["review"], inplace=True)

In [9]:
df

,label,clean_text
0,1,one of the other reviewers has mentioned that ...
1,1,a wonderful little production the filming tech...
2,1,i thought this was a wonderful way to spend ti...
3,0,basically theres a family where a little boy j...
4,1,petter matteis love in the time of money is a ...
...,...,...
49995,1,i thought this movie did a down right good job...
49996,0,bad plot bad dialogue bad acting idiotic direc...
49997,0,i am a catholic taught in parochial elementary...
49998,0,i am going to have to disagree with the previo...


In [10]:
# Séparer les données en ensembles d'entraînement et de test
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [11]:
# Sélectionner un sous-ensemble aléatoire de N échantillons pour l'entraînement et le test
N = 1000
train_subset = train_df.sample(n=N, random_state=42)
test_subset = test_df.sample(n=N, random_state=42)

In [12]:
# Créer les datasets Hugging Face
train_dataset = Dataset.from_pandas(train_subset)
test_dataset = Dataset.from_pandas(test_subset)

In [13]:
train_dataset

Dataset({
    features: ['label', 'clean_text', '__index_level_0__'],
    num_rows: 1000
})

In [14]:
test_dataset

Dataset({
    features: ['label', 'clean_text', '__index_level_0__'],
    num_rows: 1000
})

# F****inetuning complet****

DistilBERT (de "Distilled BERT") est une version plus petite et plus rapide du modèle BERT (Bidirectional Encoder Representations from Transformers) développé par Hugging Face. Il offre des performances proches de BERT avec une taille réduite et une vitesse accrue.

In [15]:
import torch
# Importe PyTorch, une bibliothèque pour les réseaux de neurones profonds et l'apprentissage automatique.

import pandas as pd
# Importe pandas, une bibliothèque pour la manipulation et l'analyse de données, souvent utilisée pour manipuler des tableaux de données.

from datasets import Dataset, DatasetDict
# Importe 'Dataset' et 'DatasetDict' de la bibliothèque 'datasets' de Hugging Face, utilisés pour gérer et préparer des ensembles de données pour l'entraînement de modèles.

from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, TrainingArguments, Trainer,  EarlyStoppingCallback
# Importe plusieurs classes de la bibliothèque 'transformers' de Hugging Face:
# - DistilBertTokenizer : pour convertir du texte en tokens compréhensibles par DistilBERT.
# - DistilBertForSequenceClassification : une version de DistilBERT préparée pour la classification de séquences (par exemple, classification de texte).
# - TrainingArguments : pour configurer les paramètres d'entraînement.
# - Trainer : pour orchestrer le processus d'entraînement du modèle.

import torch.optim as optim
# Importe le sous-module 'optim' de PyTorch, qui contient divers algorithmes d'optimisation utilisés pour mettre à jour les poids du réseau pendant l'entraînement, tels que SGD, Adam, etc.

In [16]:
# Paramètres d'entraînement
model_checkpoint = "distilbert-base-uncased"  # Modèle distilbert pré-entraîné
lr = 2e-5  # Taux d'apprentissage
batch_size = 8  # Taille du lot
num_epochs = 5  # Nombre d'époques

Dans le domaine du traitement du langage naturel (NLP), le "padding" joue un rôle crucial dans la préparation des données pour l'entraînement de modèles. Voici quelques détails clés :

Concept de Padding en NLP Uniformité des Longueurs de Séquences : Les modèles de NLP, en particulier ceux basés sur des réseaux de neurones, nécessitent que toutes les entrées (phrases, paragraphes, etc.) dans un lot de données (batch) aient la même longueur pour un traitement efficace. Cependant, dans la réalité, les séquences de mots varient en longueur. Ajout de Valeurs de Remplissage : Pour résoudre ce problème, le padding est utilisé pour "remplir" les séquences plus courtes avec des valeurs neutres (souvent des zéros ou un token spécifique) pour qu'elles correspondent à la longueur de la séquence la plus longue dans le lot.

In [17]:
# Charger le tokenizer et le modèle pré-entraîné
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
train_encodings = tokenizer(train_dataset['clean_text'], truncation=True, padding=True, return_tensors='pt')
test_encodings = tokenizer(test_dataset['clean_text'], truncation=True, padding=True, return_tensors='pt')

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [18]:
# Étape 2 : Création des datasets PyTorch
train_datasetb = Dataset.from_dict({
    # 'input_ids' : Contient les identifiants de tokens obtenus après le processus de tokenisation du texte. 
    # Ces identifiants sont nécessaires pour que le modèle comprenne et traite le texte.
    'input_ids': train_encodings['input_ids'],

    # 'attention_mask' : Un masque d'attention qui indique au modèle quels tokens dans 'input_ids' sont significatifs 
    # (c'est-à-dire non-padding) et doivent être pris en compte lors du traitement.
    'attention_mask': train_encodings['attention_mask'],

    # 'labels' : Les étiquettes associées à chaque exemple d'entraînement. Ces étiquettes sont utilisées pour l'apprentissage supervisé,
    # où le modèle apprend à associer les entrées à ces étiquettes correctes.
    # 'torch.tensor()' est utilisé pour convertir la liste des étiquettes en tensor PyTorch, 
    # ce qui est nécessaire pour le traitement avec PyTorch.
    'labels': torch.tensor(train_dataset['label'])
})

test_datasetb = Dataset.from_dict({
    'input_ids': test_encodings['input_ids'],
    'attention_mask': test_encodings['attention_mask'],
    'labels': torch.tensor(test_dataset['label'])
})

# Création d'un DatasetDict
datasetb = DatasetDict({
    'train': train_datasetb,
    'test': test_datasetb
})

In [19]:
# Création d'un modèle de classification de séquences basé sur DistilBERT
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [20]:
# Affichage du nombre de paramètres du modèle
num_params = sum(p.numel() for p in model.parameters())
print(f"Nombre de paramètres du modèle : {num_params}")

# Afficher l'architecture du modèle BERT
print(model)

Nombre de paramètres du modèle : 66955010
DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
           

In [21]:
# Étape 4 : Définition des paramètres d'entraînement
# Configuration de l'optimiseur pour l'entraînement du modèle
optimizer = optim.AdamW(model.parameters(), lr=lr)
# Définition de la fonction de perte pour l'entraînement du modèle
loss_fn = torch.nn.CrossEntropyLoss()

In [22]:
# Importer la métrique pour l'évaluation
accuracy = evaluate.load("accuracy")

In [23]:
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=1)
    accuracy_value = accuracy.compute(predictions=predictions, references=labels)["accuracy"]
    return {"accuracy": accuracy_value}

In [24]:
# Définir la correspondance entre les identifiants de classe et les étiquettes
id2label = {0: "Négatif", 1: "Positif"}
label2id = {"Négatif": 0, "Positif": 1}

In [25]:
liste_texte = [
    "This movie was a masterpiece.",
    "I was blown away by the acting.",
    "It's a classic that everyone should watch.",
    "The plot was confusing and hard to follow.",
    "The special effects were top-notch.",
    "I couldn't stop laughing throughout the movie.",
    "The soundtrack was incredible.",
    "It's a total waste of time.",
    "I'm still thinking about that ending.",
    "I wouldn't recommend it to anyone."
]

liste_texte = [clean_text(x) for x in liste_texte]

print("Prédictions du modèle non entraîné:")
print("-------------------------------------")
text = []
label = []
for texte in liste_texte:
    # Tokenizer le texte
    inputs = tokenizer.encode(texte, return_tensors="pt")
    # Calculer les logits
    logits = model(inputs).logits
    # Convertir les logits en étiquette
    predictions = torch.argmax(logits)

    text.append(texte)
    label.append(id2label[predictions.tolist()])

    print(texte + " - " + id2label[predictions.tolist()])

Prédictions du modèle non entraîné:
-------------------------------------
this movie was a masterpiece - Positif
i was blown away by the acting - Positif
it is a classic that everyone should watch - Positif
the plot was confusing and hard to follow - Positif
the special effects were topnotch - Positif
i could not stop laughing throughout the movie - Positif
the soundtrack was incredible - Positif
it is a total waste of time - Positif
i am still thinking about that ending - Positif
i would not recommend it to anyone - Positif


In [26]:
# Create DataFrame for model pre-trained
data = pd.DataFrame({"text": text, "label": label})

# Afficher les premières lignes du DataFrame
data.head()

,text,label
0,this movie was a masterpiece,Positif
1,i was blown away by the acting,Positif
2,it is a classic that everyone should watch,Positif
3,the plot was confusing and hard to follow,Positif
4,the special effects were topnotch,Positif


In [27]:
# Étape 5 : Définition des arguments d'entraînement avec Transformers

training_args = TrainingArguments(
    output_dir=model_checkpoint + "-fullfinetuned-text-classification",  # Le répertoire où les résultats de l'entraînement seront sauvegardés
    run_name="full_finetuning_experiment",
    learning_rate=lr,  # Taux d'apprentissage pour l'optimisation du modèle
    per_device_train_batch_size=batch_size,  # Taille du lot d'entraînement par périphérique (GPU ou CPU)
    per_device_eval_batch_size=batch_size,  # Taille du lot d'évaluation par périphérique (GPU ou CPU)
    num_train_epochs=num_epochs,  # Nombre d'époques d'entraînement (combien de fois le modèle parcourt l'ensemble de données)
    weight_decay=0.01,  # Terme de régularisation pour contrôler le poids de la pénalisation dans la fonction de perte
    evaluation_strategy="epoch",  # Stratégie d'évaluation, ici "epoch" signifie évaluer à la fin de chaque époque
    save_strategy="epoch",  # Stratégie de sauvegarde du modèle, ici "epoch" signifie sauvegarder le modèle à la fin de chaque époque
    load_best_model_at_end=True,  # Charger le meilleur modèle à la fin de l'entraînement
    metric_for_best_model="accuracy",  # Utilise une métrique pour déterminer le meilleur modèle
    greater_is_better=True,  # Si la métrique doit être maximisée
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [28]:
# Étape 6 : Entraînement du modèle avec Trainer de Transformers
trainerfull = Trainer(
    model=model,
    args=training_args,
    train_dataset=datasetb['train'],
    eval_dataset=datasetb['test'],
    compute_metrics=compute_metrics,  # Utiliser la fonction de calcul des métriques
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]  # Stop après 2 époques sans amélioration

)

In [29]:
import os
os.environ["WANDB_DEBUG"] = "true"  # Active les logs de debug
os.environ["WANDB_CONSOLE"] = "wrap"  # Affiche les logs directement dans la console

In [30]:
import wandb

wandb.init(mode="disabled")  # Désactive WandB
trainerfull.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.307767,0.885000
2,No log,0.407486,0.875000
3,No log,0.524644,0.881000


TrainOutput(global_step=375, training_loss=0.2567698771158854, metrics={'train_runtime': 115.3153, 'train_samples_per_second': 43.359, 'train_steps_per_second': 5.42, 'total_flos': 397402195968000.0, 'train_loss': 0.2567698771158854, 'epoch': 3.0})

In [31]:
# Sauvegarde du modèle
trainerfull.save_model("trainer_full")

In [32]:
# Déterminer automatiquement l'appareil disponible
device = "cuda" if torch.cuda.is_available() else "cpu"

# Passer le modèle sur l'appareil approprié
model.to(device)

# Afficher les prédictions du modèle entraîné
print("Prédictions du modèle entraîné :")
print("---------------------------------")

label = []
# Parcourir la liste des textes à prédire
for texte in liste_texte:
    # Tokenizer le texte
    inputs = tokenizer.encode(texte, return_tensors="pt").to(device)  # Utiliser 'cuda' si dispo, sinon 'cpu'

    # Obtenir les logits du modèle pour le texte
    logits = model(inputs).logits

    # Prédire la classe en choisissant l'indice du logit le plus élevé
    predictions = torch.max(logits, 1).indices

    label.append(id2label[predictions.item()])

    # Afficher le texte et l'étiquette prédite
    print(texte + " - " + id2label[predictions.tolist()[0]])


Prédictions du modèle entraîné :
---------------------------------
this movie was a masterpiece - Positif
i was blown away by the acting - Négatif
it is a classic that everyone should watch - Positif
the plot was confusing and hard to follow - Négatif
the special effects were topnotch - Positif
i could not stop laughing throughout the movie - Négatif
the soundtrack was incredible - Positif
it is a total waste of time - Négatif
i am still thinking about that ending - Positif
i would not recommend it to anyone - Négatif


In [33]:
# Add label to the fully fine-tuned model to compare it with the model without fine-tuning
data['label_full_finetuning'] = label

In [34]:
data.head(10)

,text,label,label_full_finetuning
0,this movie was a masterpiece,Positif,Positif
1,i was blown away by the acting,Positif,Négatif
2,it is a classic that everyone should watch,Positif,Positif
3,the plot was confusing and hard to follow,Positif,Négatif
4,the special effects were topnotch,Positif,Positif
5,i could not stop laughing throughout the movie,Positif,Négatif
6,the soundtrack was incredible,Positif,Positif
7,it is a total waste of time,Positif,Négatif
8,i am still thinking about that ending,Positif,Positif
9,i would not recommend it to anyone,Positif,Négatif


# **Layer freezing finetuning**

In [35]:
# Charger le tokenizer et le modèle pré-entraîné
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
train_encodings = tokenizer(train_dataset['clean_text'], truncation=True, padding=True, return_tensors='pt')
test_encodings = tokenizer(test_dataset['clean_text'], truncation=True, padding=True, return_tensors='pt')

In [36]:
train_datasetb = Dataset.from_dict({
    'input_ids': train_encodings['input_ids'],
    'attention_mask': train_encodings['attention_mask'],
    'labels': torch.tensor(train_dataset['label'])
})

test_datasetb = Dataset.from_dict({
    'input_ids': test_encodings['input_ids'],
    'attention_mask': test_encodings['attention_mask'],
    'labels': torch.tensor(test_dataset['label'])
})

In [37]:
# Modèle DistilBERT pour la classification de séquences
modelf = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [38]:
# Geler certaines couches du modèle
for param in modelf.base_model.parameters():
    param.requires_grad = False

In [39]:
# Définir les arguments d'entraînement pour le Trainer de Hugging Face
training_args = TrainingArguments(
    # Répertoire où les résultats d'entraînement (modèles, configurations, etc.) seront sauvegardés.
    output_dir="./freezdistilbert_finetuned",

    # Nombre d'exemples par lot (batch) pour l'entraînement, défini pour chaque périphérique (GPU/CPU).
    per_device_train_batch_size=8,

    # Nombre d'exemples par lot pour l'évaluation, défini pour chaque périphérique.
    per_device_eval_batch_size=8,

    # Stratégie d'évaluation à utiliser. Ici, "steps" signifie que l'évaluation aura lieu à intervalles réguliers de pas d'entraînement.
    evaluation_strategy="steps",

    # Nombre de pas d'entraînement à effectuer avant de réaliser une évaluation.
    eval_steps=100,

    # Nombre de pas d'entraînement après lesquels le modèle sera sauvegardé.
    save_steps=100,

    # Nombre total d'époques d'entraînement à réaliser.
    num_train_epochs=5,

    # Taux d'apprentissage initial à utiliser pour l'optimiseur.
    learning_rate=2e-5,

    # Limite du nombre total de sauvegardes de checkpoints à conserver. 
    # Ici, seul le checkpoint le plus récent sera conservé.
    save_total_limit=1
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [40]:
# Importer la métrique d'exactitude pour l'évaluation

accuracy = evaluate.load("accuracy")

In [41]:
# Fonction pour calculer l'exactitude
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=1)
    
    # Calcul de l'exactitude
    accuracy = np.mean(predictions == labels)

    return {"accuracy": accuracy}

In [42]:
from transformers import DataCollatorWithPadding  # Assurez-vous d'avoir cette ligne d'importation


# Créer un Trainer
trainerfreezing = Trainer(
    model=modelf,  # Utilisez le modèle DistilBERT que vous avez défini précédemment
    args=training_args,  # Les paramètres d'entraînement définis précédemment
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),  # Gère le padding des données lors de l'entraînement
    compute_metrics=compute_metrics,  # Fonction pour calculer les métriques d'évaluation
    train_dataset=train_datasetb,  # L'ensemble de données d'entraînement
    eval_dataset=test_datasetb  # L'ensemble de données d'évaluation
)

In [43]:
# Entraîner le modèle
trainerfreezing.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Step,Training Loss,Validation Loss,Accuracy
100,No log,0.672690,0.676000
200,No log,0.651440,0.753000
300,No log,0.638345,0.758000
400,No log,0.629243,0.759000
500,0.649600,0.622893,0.763000
600,0.649600,0.620810,0.758000


TrainOutput(global_step=625, training_loss=0.6431162475585938, metrics={'train_runtime': 98.0366, 'train_samples_per_second': 51.001, 'train_steps_per_second': 6.375, 'total_flos': 662336993280000.0, 'train_loss': 0.6431162475585938, 'epoch': 5.0})

In [44]:
# Déterminer automatiquement l'appareil disponible
device = "cuda" if torch.cuda.is_available() else "cpu"

# Passer le modèle sur l'appareil approprié
model.to(device)

# Afficher les prédictions du modèle entraîné
print("Prédictions du modèle entraîné :")
print("---------------------------------")

label = []
# Parcourir la liste des textes à prédire
for texte in liste_texte:
    # Tokenizer le texte
    inputs = tokenizer.encode(texte, return_tensors="pt").to(device)  # Utiliser 'cuda' si dispo, sinon 'cpu'

    # Obtenir les logits du modèle pour le texte
    logits = modelf(inputs).logits

    # Prédire la classe en choisissant l'indice du logit le plus élevé
    predictions = torch.max(logits, 1).indices

    label.append(id2label[predictions.item()])

    # Afficher le texte et l'étiquette prédite
    print(texte + " - " + id2label[predictions.tolist()[0]])


Prédictions du modèle entraîné :
---------------------------------
this movie was a masterpiece - Positif
i was blown away by the acting - Positif
it is a classic that everyone should watch - Positif
the plot was confusing and hard to follow - Positif
the special effects were topnotch - Positif
i could not stop laughing throughout the movie - Positif
the soundtrack was incredible - Positif
it is a total waste of time - Négatif
i am still thinking about that ending - Positif
i would not recommend it to anyone - Positif


In [45]:
data['label_model_freezing'] = label

In [46]:
data.head(10)

,text,label,label_full_finetuning,label_model_freezing
0,this movie was a masterpiece,Positif,Positif,Positif
1,i was blown away by the acting,Positif,Négatif,Positif
2,it is a classic that everyone should watch,Positif,Positif,Positif
3,the plot was confusing and hard to follow,Positif,Négatif,Positif
4,the special effects were topnotch,Positif,Positif,Positif
5,i could not stop laughing throughout the movie,Positif,Négatif,Positif
6,the soundtrack was incredible,Positif,Positif,Positif
7,it is a total waste of time,Positif,Négatif,Négatif
8,i am still thinking about that ending,Positif,Positif,Positif
9,i would not recommend it to anyone,Positif,Négatif,Positif
